In [1]:
import os
sparkFile = os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')
exec(compile(open(sparkFile, "rb").read(), sparkFile, 'exec'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.0
      /_/

Using Python version 3.5.4 (default, Oct 27 2017 11:48:53)
SparkSession available as 'spark'.


# The dataset

We will work with the NIPS dataset of the [Bag of Words Data Set](https://archive.ics.uci.edu/ml/datasets/bag+of+words). It consists of two files:

- `docword.nips.txt` contain shingles of the document. Contains the count of each words in each document. Importantly, stop words were removed, and only words appearing more than 10 times are kept.
- `vocab.nips.txt` contains all the used words.

In [2]:
vocabulary = [ word for word in map(lambda x: x.strip(), open("data/vocab.nips.txt").readlines()) ]

# check some stop words
print('the' in vocabulary or 'a' in vocabulary or 'to' in vocabulary)

# example of shingles
print(vocabulary[:20])

False
['a2i', 'aaa', 'aaai', 'aapo', 'aat', 'aazhang', 'abandonment', 'abbott', 'abbreviated', 'abcde', 'abe', 'abeles', 'abi', 'abilistic', 'abilities', 'ability', 'abl', 'able', 'ables', 'ablex']


Our goal in this practice will be to efficiently find related documents by using local-sensitivity hashing.

# Local-sensitivity hashing

## Shingling: the documents as sets

The document was already shingled, so we will be just pre-process it.

In [3]:
C = sc.textFile ("data/docword.nips.txt") \
        .map(lambda line: line.split()) \
        .filter(lambda line: len(line) == 3) \
        .map(lambda y: (y[0], int(y[1]) - 1)) \
        .groupByKey()

C is an RDD that represents the characteristic matrix of the dataset. A pair (K,V), represent the rows K of column K, which are the only non-zero values. The method we are implementing does not care about how many times an element appears.

## Minhash: getting document signatures

We define the function used to get the document signatures. We will do 1000 random pseudo-permutations, and for each of them, select the first row that is not 0.

In [4]:
import random

SIGNATURE_SIZE = 100

A = random.sample(range(1,1500), SIGNATURE_SIZE)
B = random.sample(range(1,1500), SIGNATURE_SIZE)

def min_hash(a, b, sig):
    hashes = [((a * x) + b) % len(vocabulary) for x in sig]
    return min(hashes)

def get_signature(p):
    doc,words = p
    signature = [ min_hash(a, b, words) for a,b in zip(A,B) ]
    return((doc, signature))

## LSH: getting candidate pairs

We will chunk each of the signatures and hash each of the resulting bands. Two bands from two documents falling in the same bucket is unlikely enough to be considered interesting for further study.

In [5]:
NUM_BANDS = 10

def chunk(l, n):
    l = [ x for x in l ]
    for i in range(0, len(l), int(len(l)/n)):
        yield frozenset(l[i:i + n])
        
def hash_bands(p):
    doc,sig = p   
    bands = [ ((i, hash(b)), [doc]) for i,b in enumerate(chunk(sig, NUM_BANDS)) ]
    return bands

Putting it altogether...

In [6]:
candidates = C \
    .map(get_signature) \
    .flatMap(hash_bands) \
    .reduceByKey(lambda a, b: a + b) \
    .filter(lambda v: len(v[1]) > 1) \
    .collect()

... we get ...

In [7]:
candidates

[((6, -8630840649184911538), ['820', '1497'])]

... a single candidate.

# Difficulties encountered

- PySpark is painful to debug; possibly Spark is better in that regard, as you have access to the full stack natively.
    - Errors produce many uninformative lines (full stack, Java + Python). Additionally, because of the way functions are `pickled`, it might not be clear where the error is.
    - Hard to attach a PDB. My best solution: download the notebook as a Python script, replacing
    ```
    import os
    sparkFile = os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')
    exec(compile(open(sparkFile, "rb").read(), sparkFile, 'exec'))
    ```
    by
    ```
    from pyspark import SparkContext
    sc = SparkContext("local", "App Name")
    ```
    and run with `python -m pdb` to debug the demon. I didn´t need to debug the workers; God forbid you ever have to debug on a cluster environment.
    - Difficult to access intermediate variables.
- Running PySpark/Spark in proxied networks might not be straightforward.
- Lazy evaluation leads to some counter-intuitive results. For example, once a faulty command is run, and it leads to an error, its corrected version will also lead to the same error. 